In [ ]:
from utils import *
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Segoe UI Symbol','simHei','Arial','sans-serif']

## Load Data

In [ ]:
years = [2016,2017]
resampling_period = '10T'  # 60 minutes

original_data = load_resample_data(years, resampling_period)

# Load Windfarms.xlsx
windfarms = pd.read_excel('NRGstreamData/WindFarms.xlsx').sort_values(by='Asset ID').reset_index(drop = True)
windfarms_names = original_data.columns.tolist()    

### Normalize data = Capacity Factor (optional) 

In [ ]:
normalized_data = original_data / windfarms['Capacity'].values

normalized_data.head(3)

### Decide what data to use for further analysis (Original data | Normalized data | clustered_data)

In [ ]:
sum_capacities_each_row = np.array(~original_data.isnull().values * windfarms['Capacity'].values).sum(axis=1)
normalized_data['Total'] = original_data.sum(axis=1)/ sum_capacities_each_row

clustered_data['Total'] = normalized_data['Total']

original_data['Total'] = original_data.sum(axis=1)

In [ ]:
# #### Be catious !
data = original_data  # Normalized_data  | Original_data
data.head(3)

In [ ]:
CF_total = data['Total'].values #* 100

In [ ]:
class prob_matrix_rapid_change:
    
    def __init__(self, array, t_windows = [2,3], delta_CFs = [5, 10], kind = 'RampUp'):
        self.kind = kind
        self.array = array
        self.time_windows = t_windows
        self.delta_CFs = delta_CFs
        
    def comp_prob_matrix_rapid_change(self):
        prob_mat = np.zeros((len(self.time_windows),len(self.delta_CFs)))
    
        for i,time_window in enumerate(self.time_windows):
            print(i, end=" ")

            # a = np.diff(array, time_window)
            a = self.array[time_window:] - self.array[:-time_window]

            for j,delta_CF in enumerate(self.delta_CFs):
                if self.kind == 'RampUp':
                    n_happened = sum(a > delta_CF) 
                elif self.kind == 'RampDown':
                    n_happened = sum(a < -delta_CF) 

                p = n_happened / len(a)

                prob_mat[i,j] = p
                
        self.prob_mat = prob_mat
        
        return prob_mat
    
    def plot_imageshow(self):
        data_here = self.prob_mat
        d1 = data_here.shape[0]
        d2 = data_here.shape[1]
        x_leg = 1 * c
        y_leg = 5

        fig, ax = plt.subplots(figsize=(5,4),dpi=110)
        cax = ax.imshow(data_here.transpose(), aspect='auto', cmap='YlGnBu') #norm=matplotlib.colors.LogNorm()

        ax.set_xticks(np.arange(0, d1,x_leg))
        ax.set_xticklabels(time_windows[0::x_leg].astype('int'), fontsize=9)

        y_ticks_positions = np.arange(0, d2, y_leg)
        # y_ticks_positions[-1] = y_ticks_positions[-1] + 1
        ax.set_yticks(y_ticks_positions) #[-0.5,  4.5,  9.5, 14.5, 19.5, 24.5, 30.5]
        ax.set_yticklabels(delta_Capacity[0::y_leg],fontsize=9)  # delta_Capacity or delta_CFs

        # ax.spines['left'].set_position(('data', 0.0))

        # Add colorbar, make sure to specify tick locations to match desired ticklabels
        cbar = fig.colorbar(cax)
        # cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

        ax.set_xlabel('Time window (hours)',fontsize=10)
        ax.set_ylabel('$\Delta$P (MW)',fontsize=10)

        fig.tight_layout()
        fig.savefig('{}_in_{}hours_upto{}p.jpg'.format(kind,n_hours,max(delta_Capacity)), dpi = 300, bbox_inches='tight')

In [ ]:
c = int(60 / int(resampling_period[:-1]))
n_hours = 2
kind = 'RampDown'
t_windows = np.arange(c, n_hours *  c + 1)
delta_CFs = np.arange(100,301,100)

prob_matrix_ramp_down = prob_matrix_rapid_change(CF_total, t_windows, delta_CFs, kind)
prob_matrix_ramp_down_matrix = prob_matrix_ramp_down.comp_prob_matrix_rapid_change()
prob_matrix_ramp_down.plot_imageshow()

In [ ]:
c = int(60 / int(resampling_period[:-1]))
n_hours = 2
kind = 'RampUp'
t_windows = np.arange(c, n_hours *  c + 1)
delta_CFs = np.arange(100,301,100)

prob_matrix_ramp_up = prob_matrix_rapid_change(CF_total, t_windows, delta_CFs, kind)
prob_matrix_ramp_up_matrix = prob_matrix_ramp_down.comp_prob_matrix_rapid_change()
prob_matrix_ramp_up.plot_imageshow()